In [1]:
import dxfgrabber
import math
import numpy as np

In [2]:
dxf_path = '../客流点位图-F.dxf'
dxf = dxfgrabber.readfile(dxf_path)

In [3]:
print(dxf.layers.names())

['--盲道', '0', '0-BAFFLE WALL', '0-车库集水坑', '000类别', '00常熟铺位号', '02_PART_AXES', '03__ASM_ALL_CURVES', '04_PART_CURVES', '0928修改', '0CABR-铝板剖面', '0面积', '10-18修改', '160 80 6', '252', '3f$0$A-土建墙（含管井、立管）', 'A-AlumLine铝板看线', 'A-Alum铝板', 'A-Anno-Dims', 'A-Anno-Note', 'A-ELE-SIGN', 'A-FURN', 'A-Glaze玻璃', 'A-Index(建筑索引线剖切号线)', 'A-Insulation保温', 'A-Pub-Text(文字标注)', 'A-SCRN-HIDD', 'A-STRS', 'A-StelStruLine龙骨看线', 'A-TexturePaint真石漆', 'A-WALL-FNSH', 'A-WallLine墙看线', 'A-space(天正房间名称', 'A-土建墙填充', 'A-土建墙（含管井、立管）', 'A-窗', 'A-立面-窗户', 'A-置物台', 'AX-B02-P04$0$A-FURN', 'AX-P05-WALL$0$A-Wind', 'AX-SYJ-PROOF-COMP$0$0CABR-铝板剖面', 'AXIS', 'BALCONY', 'BS-TEXT-文字', 'BZ', 'Bounds', 'C-顶面造型(平面显示)', 'CLIEC', 'COLU', 'COLUMN', 'CURTWALL', 'DEFAULT_2', 'DIM', 'DIM-文字', 'DIM_ELEV', 'DIM_IDEN', 'DIM_SYMB', 'DIM_标注_A', 'DOOR_FIRE', 'DOOR_FIRE_TEXT', 'DUCT-加压送风管立管', 'DUCT-加压送风管端线', 'DUCT-回风管立管', 'DUCT-回风管端线', 'Defpoints', 'Dimensions', 'EL-标注', 'EQUIP_地漏', 'EQUIP_消火栓', 'EQUIP_雨水斗', 'EVTR', 'E_1', 'F-地坪分割线', 'GC', 'HANDRAIL

In [4]:
ids = [e for e in dxf.entities if e.layer == 'ID' and (e.dxftype == 'TEXT' or e.dxftype == 'MTEXT')]
types0 = set([e.dxftype for e in ids])
print(len(ids),types0)

262 {'MTEXT', 'TEXT'}


In [5]:
bounds = [e for e in dxf.entities if e.layer == 'Bounds' and e.dxftype == 'LWPOLYLINE']
types0 = set([e.dxftype for e in bounds])
print(len(bounds),types0)

271 {'LWPOLYLINE'}


In [6]:
len([e for e in bounds if e.dxftype == 'LWPOLYLINE'])

271

In [7]:
ids_t = [e for e in ids if e.dxftype == 'TEXT']

In [14]:
dict_attachment_point = {1:[-1,1],
                         2:[0,1],
                         3:[1,1],
                         4:[-1,0],
                         5:[0,0],
                         6:[1,0],
                         7:[-1,-1],
                         8:[0,-1],
                         9:[1,-1]}

In [22]:
def get_mtext_center(e,stylewidth):
    rows = np.array([len(line) for line in e.lines() if line != ''])
    x = rows.mean()/2 * stylewidth * e.height*0.8 #调试系数0.8
    lines = np.array([i for i,line in enumerate(e.lines()) if line != ''])
    y = (lines.mean()+0.5) * e.height*1.6 #调试系数1.6
    attach = dict_attachment_point[e.attachment_point]
    center_x = -x * attach[0]+e.insert[0]
    center_y = -y * attach[1]+e.insert[1]
    return center_x,center_y

In [16]:
def get_text_center(e):
    height = e.height  
    center_x = e.align_point[0]
    center_y = e.align_point[1] + 0.5 * height
    return center_x,center_y

In [33]:
def get_centerpoint(bounds):
    area = 0.0
    x,y = 0.0,0.0
 
    a = len(bounds)
    for i in range(a):
        x0 = bounds[i][0]
        y0 = bounds[i][1]
        x1 = bounds[i-1][0]
        y1 = bounds[i-1][1]
 
        fg = (x0*y1 - x1*y0)/2.0
 
        area += fg
        x += fg*(x0+x1)/3.0
        y += fg*(y0+y1)/3.0
 
    x = x/area
    y = y/area
    return x,y

In [55]:
def match(ics,bcs):
    miniindex = []
    minidistance = []
    for i in range(len(ics)):
        ic = ics[i]
        distance = [abs(ic[0]-bc[0])+abs(ic[1]-bc[1]) for bc in bcs]
        index = distance.index(min(distance))
        dis = min(distance)
        if index in miniindex:
            i = miniindex.index(index)
            if minidistance[i] > dis:
                miniindex[i] = -1
                minidistance[i] = -1
                miniindex.append(index)
                minidistance.append(dis)
            else:
                miniindex.append(-1)
                minidistance.append(-1)
        else:
            miniindex.append(index)
            minidistance.append(dis)
    return miniindex, minidistance


In [50]:
ids_center = np.array([get_mtext_center(e,dxf.styles.get(e.style).width) if e.dxftype == 'MTEXT' else get_text_center(e) for e in ids ])
ids_center

array([[2160831.9051126 , 6157921.42193293],
       [2171431.30320349, 6158202.74277007],
       [2090676.27991509, 6130394.82632446],
       [2103559.13862518, 6129298.43946797],
       [2112618.17580785, 6129444.17675253],
       [2129014.36043283, 6131132.40109273],
       [2137493.54651533, 6130394.82632446],
       [2145204.90844321, 6128429.95462832],
       [2153894.21318127, 6128632.89032054],
       [2169180.07501517, 6130229.86811486],
       [2179102.69169268, 6130151.02907917],
       [2227745.99322079, 6158068.01925251],
       [2226501.88667955, 6165243.24140121],
       [2179730.62242895, 6213154.31437383],
       [2171071.03705231, 6213374.76096654],
       [2125483.25294392, 6203236.89415852],
       [2109064.14199242, 6193237.46483128],
       [2103217.75999257, 6186114.90667795],
       [2088752.88876731, 6178025.35846131],
       [2067344.24828998, 6165089.19203805],
       [2179136.04651533, 6185940.82632446],
       [2170675.04651533, 6185940.82632446],
       [21

In [49]:
bounds_center = np.array([get_centerpoint(bound) for bound in bounds])
bounds_center

array([[2170693.11184272, 6159249.5193964 ],
       [2212254.39425503, 4990096.65852169],
       [2274451.02115907, 5371302.31422922],
       [2160203.0539122 , 6159229.98876968],
       [2154237.05098696, 6130751.73405989],
       [2169592.43535549, 6132014.7575332 ],
       [2178650.61962488, 6129472.3523403 ],
       [2103218.98864607, 6129386.70932359],
       [2112615.83819617, 6130332.61531669],
       [2134104.56093545, 6159241.56929285],
       [2120425.68273348, 6156023.71944962],
       [2179218.61743939, 6159249.08953484],
       [2196309.66664039, 6216260.23259896],
       [2194319.5568448 , 6173560.21002883],
       [2190867.43066223, 6181680.63427968],
       [2188630.85871079, 6189541.18639445],
       [2229426.28149919, 6189229.84585758],
       [2095611.70126873, 6154473.37987337],
       [2224020.09731297, 6164714.06508265],
       [2225628.6077942 , 6156746.39161361],
       [2225628.94048091, 6148502.21857764],
       [2103669.61550538, 6185234.62135161],
       [21

In [59]:
id_,dis_ = match(ids_center,bounds_center)
print(id_)

[3, 0, 220, 7, 8, 40, 39, 24, 4, 5, 6, 19, 18, 111, -1, 217, 22, 21, 25, 26, 33, 34, 35, 36, 37, 28, 29, 222, 13, 20, 266, 23, 11, 12, 17, 30, 9, 15, 110, 14, 16, 31, 10, 32, -1, 38, 27, 221, 41, 216, 223, 219, 218, 42, 43, 49, 45, 208, 51, 58, 52, 53, 204, 55, 54, 62, 61, 48, 63, 212, 59, 56, 209, 210, 46, 47, 57, 73, 60, 66, 67, 69, 71, 70, 50, 68, 72, 104, 80, 97, 93, 102, 107, 83, 82, -1, 105, 81, 94, -1, 90, 214, 100, -1, 87, 89, 85, 101, 78, 215, 75, 86, 96, 99, 88, 77, 84, 98, 108, -1, -1, 92, 95, 91, 106, 213, -1, -1, 76, 103, 79, 109, -1, 130, 120, 128, 115, 233, 237, 133, 135, 123, 132, 231, 144, 127, 143, 124, 236, 225, 121, 134, 226, 141, 118, 119, 122, 129, 235, 267, 125, 137, 131, 140, 136, 117, 232, 138, 227, 114, 139, 269, 142, 265, 126, 268, 270, 145, -1, 234, 238, 152, 153, 171, 170, 1, 154, 156, 157, 259, 158, 160, 161, 166, 165, 164, 149, 155, 162, 151, 261, 262, 260, 258, 167, 150, 159, 148, 264, 169, 168, 174, 176, 263, 175, 182, 191, 253, 251, 250, 240, 256, 181,

In [65]:
fail = [i for i,c in enumerate(id_) if c == -1]
fail

[14, 44, 95, 99, 103, 119, 120, 126, 127, 132, 178, 258, 259, 260]

In [48]:
name = [get_text(e) for e in ids]
len(name) == len(ids_center)

True

In [46]:
def get_text(e):
    text =  ''.join(e.lines()) if e.dxftype == 'MTEXT' else e.text 
    text = text.replace(' ', '')
    text = text.split(';')[-1]
    text=  ''.join(text.split('}'))
    return text

In [30]:
a= npa[1 if i%2==0  else 0 for i in range(10)]
a

[1, 0, 1, 0, 1, 0, 1, 0, 1, 0]

In [ ]:
print(get_text_center(e))

In [ ]:

rows = np.array([len(line) for line in e.lines() if line != ''])
x = rows.mean()/2 * dxf.styles.get(e.style).width * e.height*0.8 #调试系数0.8
x 

In [ ]:
attach = dict_attachment_point[e.attachment_point]
center_x = -x * attach[0]+e.insert[0]
center_y = -y * attach[1]+e.insert[1]
(center_x,center_y)

In [ ]:
style = dxf.styles.get('微软雅黑')

In [ ]:
style.height

In [ ]:
e.height

In [ ]:
e.vertical_height

In [ ]:
e.horizontal_width

In [ ]:
e.rect_width

In [ ]:
style.height

In [ ]:
style.width

In [ ]:
style.big_font

In [ ]:
print(get_centerpoint([(1,0),(0,1),(1,1)]))

In [ ]:
lp = [get_centerpoint(lp.points) for lp in bounds]
lp

In [67]:
e = bounds[0]
e.points

[(2174908.654576039, 6166979.988761497),
 (2166497.453243152, 6166979.988761497),
 (2166497.453243528, 6151479.988761726),
 (2174158.65456439, 6151479.988770748),
 (2174158.654564201, 6151929.988782627),
 (2174908.654576054, 6151929.988782273)]